In [29]:
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pip install nltk

In [30]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) # for downloading packages
#nltk.download('punkt') # unang run lang
#nltk.download('wordnet') # unang run lang din

True

This is where the data will input. I named it "intents.json" juust for sample hehe (data is not ours, tbh acquired it from somewhere down the road)


In [31]:
import json

# Read the JSON data
with open('intents.json', 'r', errors='ignore') as f:
    data = json.load(f)

preprocessed_data = []
for intent in data['intents']:
    patterns = [pattern.lower() for pattern in intent['patterns']]
    preprocessed_data.append({
        'tag': intent['tag'],
        'patterns': patterns,
        'responses': intent['responses']
    })



**<h1>Tokenization</h1>**

In [32]:
tokenized_data = []
for intent in data["intents"]:
    tag = intent["tag"]
    patterns = intent["patterns"]
    tokenized_patterns = [nltk.word_tokenize(pattern.lower()) for pattern in patterns]
    tokenized_data.append({
        "tag": tag,
        "patterns": tokenized_patterns,
        "responses": intent["responses"],
        "context_set": intent["context_set"]
    })

**<h1>Preprocessing</h1>**  This is where i will use word net lemmatizers

In [33]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

**<h1>Handle Greetings</h1>**

In [34]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey", "hello there", "yo", "oy", "Hello LSPU")
GREETING_RESPONSES = ["hi", "hey", "im here!", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def response(user_response):
    # Initialize robo_response
    robo_response = ''

    # Calculate TF-IDF vectors
    tfidf = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf.fit_transform([pattern for intent in data["intents"] for pattern in intent["patterns"]])

    # Calculate cosine similarity between user response and patterns
    user_tfidf = tfidf.transform([user_response])
    similarities = cosine_similarity(user_tfidf, tfidf.transform([pattern for intent in data["intents"] for pattern in intent["patterns"]]))

    # Find the index of the most similar pattern
    most_similar_index = similarities.argsort()[0][-1]

    # Retrieve the corresponding response
    robo_response += data["intents"][most_similar_index]["responses"][0]  # Assuming there's only one response per pattern

    return robo_response


In [ ]:
flag=True
print("Welcome to LSPU Chatbot")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

In [5]:
import json
import nltk
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Read the JSON data
with open('intents.json', 'r', errors='ignore') as f:
    data = json.load(f)

# Preprocess the data
lemmer = nltk.stem.WordNetLemmatizer()
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# Process intents and responses
intents = []
responses = {}

for intent in data['intents']:
    for pattern in intent['patterns']:
        intents.append(pattern)
        responses[pattern] = intent['responses'][0]  # Assuming there's only one response per pattern

# Create TF-IDF vectors
tfidf = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
tfidf.fit_transform(intents)

# Define greeting responses
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up", "hey", "hello there", "yo", "oy", "hello lspu")
GREETING_RESPONSES = ["hi", "hey", "im here!", "hi there", "hello", "i am glad! you are talking to me"]

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)
# Define response function
def response(user_response):
    user_tfidf = tfidf.transform([user_response])
    similarities = cosine_similarity(user_tfidf, tfidf.transform(intents))
    most_similar_index = similarities.argsort()[0][-1]
    return responses[intents[most_similar_index]]  # Retrieve the corresponding response

# Chat loop
flag = True
print("Welcome to LSPU Chatbot")
while flag:
    user_response = input().lower()
    print("User: " + user_response)  # Print user input
    if user_response != 'bye':
        if user_response in ('thanks', 'thank you'):
            flag = False
            print("ROBO: You are welcome..")
        else:
            if greeting(user_response) is not None:
                print("ROBO: " + greeting(user_response))
            else:
                print("ROBO: " + response(user_response))
    else:
        flag = False
        print("ROBO: Bye! take care..")


Welcome to LSPU Chatbot
User: hello
ROBO: hi there
User: hello
ROBO: i am glad! you are talking to me
User: any courses available?
ROBO: Our university offers Information Technology, computer Engineering, Mechanical engineering,Chemical engineering, Civil engineering and extc Engineering.
User: what are food in canteen?
ROBO: we serve Franky, Locho, Alu-puri, Kachori, Khavsa, Thaali and many more on menu
User: a
ROBO: HODs differ for each branch, please be more specific like: (HOD it)
User: bye
ROBO: Bye! take care..
